#Importing the Libraries

In [144]:
!pip install keras
!pip install pandas
!pip install sklearn
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [130]:
import numpy as np
import pandas as pd 
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import OneHotEncoder
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import warnings
warnings.filterwarnings('ignore')

Reading the Data

In [120]:
data=pd.read_csv("/content/drive/MyDrive/ls2/train.csv")

In [121]:
data.shape

(2500, 3)

Cleaning Data

In [122]:


data.iloc[:,2] = data.iloc[:,2].apply(lambda x: x.lower())
data.iloc[:,2] = data.iloc[:,2].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))
data.iloc[:,2] = data.iloc[:,2].apply((lambda x: re.sub('[([?:|[0-9]|[$-.&+#]|[!*\(\),]', '',x)))


In [123]:
data.head(5)
tweets = data.loc[:,"tweet"]

Converting Text into Sequences

In [125]:
max_features = 2000
tokenizer = Tokenizer(num_words=max_features, split=' ')
tokenizer.fit_on_texts(tweets.values)
X = tokenizer.texts_to_sequences(tweets.values)
X = pad_sequences(X)

In [126]:
lstm_out = 196
embed_dim = 128

In [140]:
model = Sequential()
model.add(Embedding(max_features, embed_dim,input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 30, 128)           256000    
_________________________________________________________________
spatial_dropout1d_6 (Spatial (None, 30, 128)           0         
_________________________________________________________________
lstm_6 (LSTM)                (None, 196)               254800    
_________________________________________________________________
dense_6 (Dense)              (None, 2)                 394       
Total params: 511,194
Trainable params: 511,194
Non-trainable params: 0
_________________________________________________________________
None


In [141]:
Y = pd.get_dummies(data.iloc[:,1])
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.2, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(2000, 30) (2000, 2)
(500, 30) (500, 2)


Training Model

In [142]:

batch_size = 16
model.fit(X_train, Y_train, epochs = 50, batch_size=batch_size,validation_data=(X_test,Y_test))

Epoch 1/50
125/125 [==============================] - 18s 148ms/step - loss: 0.2691 - accuracy: 0.9325 - val_loss: 0.2299 - val_accuracy: 0.9300
Epoch 2/50
125/125 [==============================] - 24s 193ms/step - loss: 0.1816 - accuracy: 0.9390 - val_loss: 0.1791 - val_accuracy: 0.9340
Epoch 3/50
125/125 [==============================] - 18s 145ms/step - loss: 0.0982 - accuracy: 0.9635 - val_loss: 0.1776 - val_accuracy: 0.9380
Epoch 4/50
125/125 [==============================] - 18s 145ms/step - loss: 0.0643 - accuracy: 0.9755 - val_loss: 0.1886 - val_accuracy: 0.9380
Epoch 5/50
125/125 [==============================] - 18s 146ms/step - loss: 0.0420 - accuracy: 0.9870 - val_loss: 0.2412 - val_accuracy: 0.9400
Epoch 6/50
125/125 [==============================] - 18s 146ms/step - loss: 0.0279 - accuracy: 0.9925 - val_loss: 0.2694 - val_accuracy: 0.9320
Epoch 7/50
125/125 [==============================] - 18s 145ms/step - loss: 0.0152 - accuracy: 0.9950 - val_loss: 0.3230 - val_ac

Importing the Test Data

In [147]:
data1 = pd.read_csv("/content/drive/MyDrive/wana-interview-test-master/wana-interview-test-master/datascience/sentiment_posts.csv")


In [148]:
data1.iloc[:,1] = data1.iloc[:,1].apply(lambda x: x.lower())
data1.iloc[:,1] = data1.iloc[:,1].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))
data1.iloc[:,1] = data1.iloc[:,1].apply((lambda x: re.sub('[([?:|[0-9]|[$-.&+#]|[!*\(\),]', '',x)))

In [149]:
text_original = data1.loc[:,"text_original"]

In [151]:
max_features = 2000
tokenizer = Tokenizer(num_words=max_features, split=' ')
tokenizer.fit_on_texts(text_original.values)
X = tokenizer.texts_to_sequences(text_original.values)
X = pad_sequences(X)
result = model.predict(X)

In [173]:
res = np.argmax(result, axis = 1)
print(res)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [172]:

res = np.argmax(result, axis = 1)
for i in range(len(text_original)):
  if res[i] == 0:
    print(text_original[i], "Positive Statement", "\n")
  else:
    print(text_original[i], "Negative Statement", "\n")


it’s a question that’s plagued me for a long time (see my previous post). these people get into the best schools, have a lot of popularity/fame, high status/prestigious. does that bother you and how do you deal with it? it’s so hard for me to accept at times, leaving me a bit bitter about it. Positive Statement 

How is everyone REALLY doing today, mentally? ❤️Fabulous  🧡 Mostly Ok 💛 indifferent 💚 Mostly Not Ok 💙 Severly Upset  Positive Statement 

what scares you the most? Positive Statement 

what  that one thing that puts  a smile o your face  Positive Statement 

I'm tired I've turned into a totally different person and I'm emotionally not there for my family and friends and self anymore I want to love but every time I get close I go right back to being an empty void Positive Statement 

Sometimes, when a doctor comes in to the room, I can almost tell they've taken a quick look at my extensive medical chart and have already made their mind up that they can’t help me, before they’ve